<img align="left" style="padding-right:10px;" width="250" src="img/Marine_logo.jpg"></br><h2><center>Benthic Update Data</center></h2>

# Jupyter Notebook for Updating Database Tables

## Insert data into SQL database using Pandas

- Taking the individual excel template for each of the core data types, this script using the Pandas python library to convert that Excel data into a data frame.
- That data frame is then reshaped and adapted to comply with the schema set out in SQL server

## Notes
- Currently set to interact with server database held on local host with files held locally.
- Only the "file" and "server" variables need to ever be changed. 
- Below code will be cleaned-up and centralised on to a server hub where anyone with the appropriate permissions can run it.

## Establish connection to Database Server
Import all necessary modules which are used to interact with the different data sources and also used to convert data to work with the end Database server.
The majority of these packages are included in the standard **Pandas** implementation.
Packages not included in this will be reviewed and removed as alternatives are compliance tested.

In [ ]:
import pyodbc
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import xlrd
import sqlalchemy
from sqlalchemy import create_engine
import urllib.parse

server = r'miags02.marine.ie'
# wherever the database is going to be eg.'localhost\sqlexpress'
driver = '{SQL Server}'
# Driver & default port used to connect to the database
port = '1433'


Once the connection to the data source has been established and data frame created, the data itself can be manipulated however you want:
- In the below snippet, the data stored in the frame is renamed and reorganised to comply with the formatting of the created SQL database.
- Variables are created for items in DB but not in the dataframe.
- Superfluous columns are dropped from the data frame, but doing that here **never** affects the source data.

## Files
Remember to change what file is equal to.
    - Set it to your local machine or wherever the data you want is stored.

## Attributes

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

df = pd.read_excel(file, sheet_name='attributes')

print("Column headings:")
print(df.columns)

print(df.dtypes)


With the data frame cajoled into the required format, the SQL database is reconnected to and the data is inserted in the required table.
- Note that the data types can be adjusted here if necessary.
- Only the name of the table *has* to be changed as you go to a different table in the same database.

In [ ]:
# re-establish link to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB
df.to_sql("attribute", engine, schema="dbo", if_exists="append", index=False, chunksize=100)

print(df.dtypes)

The same process can then be undertaken to populate the other tables in the database with different data sources.
- Note that minor changes may be required to the data source and types as necessary.
- In this below implementation, the entire script is run at once.

## Species Count

To rearrange the content of the original species data file so it can be easily viewed and ingested into the database, click [here](Benthic excel sheet R.IPYNB).

**Note**: You *must* have the R Kernel installed to run this linked notebook.

Follow the instructions [here](https://medium.com/@kyleake/how-to-install-r-in-jupyter-with-irkernel-in-3-steps-917519326e41) if you don't have it installed.

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

dfAtt = pd.read_excel(file, sheet_name='species')

print("Column headings:")
print(dfAtt.columns)
print(dfAtt.head(n=5))
print(dfAtt.dtypes)

# establish connection to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB
dfAtt.to_sql("speciesCount", engine, schema="dbo", if_exists="append", index=False, chunksize=100)

Errors can occur at insertion but generally these are down to cast errors and SQL being unable to reconcile a letter to a number type.
If you have an error, check your data, make sure there's nothing crazy there and rerun.

## IQI SRT

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

df = pd.read_excel(file, sheet_name='iqiSampleResult')

print("Column headings:")
print(df.columns)
print(df.head(n=5))
print(df.dtypes)

# establish connection to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB
df.to_sql("iqiSampleResult", engine, schema="dbo", if_exists="append", index=False, chunksize=10)

## IQI Waterbody RT

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

df = pd.read_excel(file, sheet_name='iqiWaterBodyResult')

print("Column headings:")
print(df.columns)
print(df.head(n=5))
print(df.dtypes)

# establish connection to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB
df.to_sql("iqiWaterbodyResult", engine, schema="dbo",
          if_exists="append", index=False, chunksize=100)

## PSA

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

df = pd.read_excel(file, sheet_name='psa')

print("Column headings:")
print(df.columns)
print(df.head(n=5))

# establish connection to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB, changing datatypes as necessary
df.to_sql("psa", engine, schema="dbo",
          if_exists="append", index=False, chunksize=10,
          dtype={'sample_id': sqlalchemy.types.NVARCHAR(length=15),
                 'Medium_Gravel_greater_8mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Fine_Gravel_4_to_8mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Very_fine_Gravel_2_to_4mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Very_coarse_sand_1_to_2mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Coarse_sand_half_to_1mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Medium_sand_quart_to_half_mm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Fine_sand_125_to_250microm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'Very_fine_sand_62_to_125microm': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'less_63microm_siltClay': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'LOI_percentage_450degrees': sqlalchemy.types.Float(precision=5, scale=5, asdecimal=True),
                 'folk_Classification_1954': sqlalchemy.types.NVARCHAR(length=254)})
print(df.dtypes)

## Ref C

In [ ]:
# here you set your URL/path for attribute table excel data
file = r'\Users\aconway\Documents\Benthic\Template.xlsx'

dfAtt = pd.read_excel(file, sheet_name='referenceCollection')

print("Column headings:")
print(dfAtt.columns)
print(dfAtt.head(n=5))
print(dfAtt.dtypes)

# establish connection to DB location
dbName = 'Benthic'
params = urllib.parse.quote('Trusted_Connection=yes;DRIVER=' + driver +
                            ';SERVER=' + server + ';DATABASE=' + dbName + ';')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# write the DataFrame to a table in the sql DB
dfAtt.to_sql("referenceCollection", engine, schema="dbo",
             if_exists="append", index=False, chunksize=100)

[Index](Index.ipynb) | [Create Tables](BenthicDatabase-CreateTables.ipynb) | [Create Database](BenthicDatabase-CreateDB.ipynb)